<a href="https://colab.research.google.com/github/RulCab/ChallengeSolvento/blob/main/challenge_solvento_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [188]:
#crear tablas


import sqlite3
conn = sqlite3.connect('test_database.db')
c = conn.cursor()

c.execute('''
CREATE TABLE IF NOT EXISTS items (
    item_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    weight REAL NOT NULL,
    price REAL NOT NULL
)''')


c.execute('DROP TABLE IF EXISTS trips')
c.execute('''
CREATE TABLE trips (
    trip_id INTEGER PRIMARY KEY AUTOINCREMENT,
    zip_codes TEXT NOT NULL,
    assigned_truck TEXT,
    planning_date TEXT NOT NULL,
    FOREIGN KEY (assigned_truck) REFERENCES trucks(plate_number)
)
''')


c.execute('''
CREATE TABLE IF NOT EXISTS trucks (
    plate_number TEXT PRIMARY KEY,
    max_weight REAL NOT NULL,
    address TEXT NOT NULL,
    working_days TEXT NOT NULL,
    available BOOLEAN NOT NULL DEFAULT TRUE
)''')


c.execute('DROP TABLE IF EXISTS purchases')

c.execute('''
CREATE TABLE IF NOT EXISTS purchases (
    purchase_id INTEGER PRIMARY KEY,
    customer_name TEXT NOT NULL,
    date TEXT NOT NULL,
    address TEXT NOT NULL,
    state TEXT NOT NULL,
    delivery_date TEXT,
    trip_id INTEGER,
    FOREIGN KEY (trip_id) REFERENCES trips(trip_id)
)''')

c.execute('''
CREATE TABLE IF NOT EXISTS purchase_items (
    purchase_id INTEGER,
    item_id INTEGER,
    quantity INTEGER,
    FOREIGN KEY (purchase_id) REFERENCES purchases(purchase_id),
    FOREIGN KEY (item_id) REFERENCES items(item_id),
    PRIMARY KEY (purchase_id, item_id)
)''')


# insertar datos en items
items_to_insert = [
    (1, 'Navigation Log Pose', 0.2, 300.00),
    (2, 'Den Den Mushi', 1.0, 150.00),
    (3, 'Tony Chopper Hat', 0.3, 15.00),
    (4, "Roronoa Zoro's Swords Set", 5.0, 500.00),
    (5, 'Straw Hat', 0.05, 20.00),
    (6, 'Devil Fruit', 0.2, 1000.00),
    (7, 'Super Heavy Item 1', 50.0, 200.00),
    (8, 'Super Heavy Item 2', 100.0, 500.00)
]
c.executemany('INSERT OR IGNORE INTO items (item_id, name, weight, price) VALUES (?, ?, ?, ?)', items_to_insert)

# insertar datos en trucks
trucks_to_insert = [
    ('MERRY GO', 210, 'Dock #0, East Blue', 'Monday,Wednesday,Friday', 1),
    ('GOING MERRY', 120, 'Dock #1, East Blue', 'Monday,Tuesday,Thursday', 1),
    ('THOUSAND SUNNY', 190, 'Dock #2, Grand Line', 'Wednesday,Friday', 1)
]
c.executemany('INSERT OR IGNORE INTO trucks (plate_number, max_weight, address, working_days, available) VALUES (?, ?, ?, ?, ?)', trucks_to_insert)

# insertar datos en purchases

purchases_to_insert = [
    (1, 'Tony Chopper', '2023-10-01', 'Doctor\'s Lab, Drum Island', 'Pending', None, 1),
    (2, 'Monkey D. Luffy', '2023-10-05', 'Thousand Sunny, Grand Line', 'OnTrip', None, 2),
    (3, 'Nico Robin', '2023-10-06', 'Archaeological Camp, Ohara', 'Delivered', '2023-10-07', 3),
    (4, 'Usopp', '2023-12-01', 'Sniper Island', 'Pending', None, 4),
    (5, 'Franky', '2023-12-01', 'Water 7', 'Pending', None, 5)
]
c.executemany('INSERT INTO purchases (purchase_id, customer_name, date, address, state, delivery_date, trip_id) VALUES (?, ?, ?, ?, ?, ?, ?)', purchases_to_insert)

# Insertar datos en purchase_items
purchase_items_to_insert = [
    (1, 3, 1),  # Tony Chopper compra
    (2, 5, 1),  # Monkey D. Luffy compra
    (2, 6, 1),  # Monkey D. Luffy compra otro ítem
    (3, 4, 2),  # Nico Robin compra
    (4, 7, 1),  # Usopp compra Super Heavy Item 1
    (5, 8, 1)   # Franky compra Super Heavy Item 2
]
c.executemany('INSERT OR IGNORE INTO purchase_items (purchase_id, item_id, quantity) VALUES (?, ?, ?)', purchase_items_to_insert)


c.execute('UPDATE trucks SET available = 1')
conn.commit()


conn.commit()

In [189]:
import sqlite3
from datetime import datetime

# Función para establecer conexión con la base de datos
def get_db_connection():
    conn = sqlite3.connect('test_database.db')
    conn.row_factory = sqlite3.Row  # facilita el acceso a las columnas por nombre
    return conn


def create_trips(trip_locations, planning_date):
    conn = get_db_connection()  # obtener conexión a la base de datos
    for location in trip_locations:  # iterar sobre cada ubicación proporcionada
        conn.execute('''
            INSERT INTO trips (zip_codes, planning_date) VALUES (?, ?)
        ''', (location, planning_date))  # insertar un nuevo viaje en la base de datos con la ubicación y la fecha de planificación
    conn.commit()
    conn.close()

def assign_trucks_to_trips(planning_date):
    conn = get_db_connection()  # obtener conexión a la base de datos
    conn.execute('BEGIN TRANSACTION;')  # comenzar una nueva transacción
    try:
        trips = conn.execute('SELECT * FROM trips WHERE planning_date = ?', (planning_date,)).fetchall()  # recuperar todos los viajes planificados para la fecha dada
        for trip in trips:  # iterar sobre cada viaje
            trip_id = trip['trip_id']  # obtener el ID del viaje
            trip_weight = conn.execute('''
                SELECT IFNULL(SUM(items.weight * purchase_items.quantity), 0) AS total_weight
                FROM purchase_items
                JOIN items ON purchase_items.item_id = items.item_id
                JOIN purchases ON purchase_items.purchase_id = purchases.purchase_id
                WHERE purchases.trip_id = ?
            ''', (trip_id,)).fetchone()['total_weight']  # calcular el peso total necesario para el viaje

            if trip_weight > 0:  # verificar si hay peso que transportar
                truck = conn.execute('''
                    SELECT plate_number FROM trucks
                    WHERE max_weight >= ? AND available = 1
                    ORDER BY max_weight ASC
                    LIMIT 1
                ''', (trip_weight,)).fetchone()  # buscar un camión disponible que pueda manejar el peso total

                if truck:  # si se encuentra un camión adecuado
                    conn.execute('UPDATE trucks SET available = 0 WHERE plate_number = ?', (truck['plate_number'],))  # marcar el camión como no disponible
                    conn.execute('UPDATE trips SET assigned_truck = ? WHERE trip_id = ?', (truck['plate_number'], trip_id))  # asignar el camión al viaje
                else:  # si no se encuentra ningún camión
                    print(f"Error: No trucks meet requirements for trip {trip_id} with required capacity {trip_weight}. Available trucks cannot handle the load.")  # imprimir error
            else:  # si no hay peso que transportar
                print(f"Warning: No purchases linked to trip {trip_id}, or items are weightless.")  # imprimir advertencia
        conn.execute('COMMIT;')  # confirmar la transacción
    except Exception as e:  # manejar excepciones
        print(f"An error occurred: {e}")  # imprimir el error
        conn.execute('ROLLBACK;')  # revertir todos los cambios si ocurre un error
    conn.close()  # cerrar la conexión a la base de datos






# Lista de lugares de One Piece como códigos de viaje
one_piece_locations = ['Sniper Island', 'Water 7', 'Sabaody Archipelago', 'Marineford', 'Impel Down']
planning_date = '2023-11-01'

# Crear viajes con nombres de lugares de One Piece
create_trips(one_piece_locations, planning_date)

# Asignar camiones a los viajes
assign_trucks_to_trips(planning_date)

Error: No trucks meet requirements for trip 4 with required capacity 50.0. Available trucks cannot handle the load.
Error: No trucks meet requirements for trip 5 with required capacity 100.0. Available trucks cannot handle the load.


In [190]:
def get_all_items():
    conn = get_db_connection()
    items = conn.execute('SELECT * FROM items').fetchall()  # recuperar todos los items de la base de datos
    conn.close()
    return [dict(item) for item in items]  # convertir cada item en un diccionario y devolver la lista

def get_all_trucks():
    conn = get_db_connection()
    trucks = conn.execute('SELECT * FROM trucks').fetchall()  # recuperar todos los camiones de la base de datos
    conn.close()
    return [dict(truck) for truck in trucks]  # convertir cada camión en un diccionario y devolver la lista


def get_all_purchases():
    conn = get_db_connection()
    purchases = conn.execute('''
        SELECT p.*, pi.item_id, pi.quantity
        FROM purchases p
        JOIN purchase_items pi ON p.purchase_id = pi.purchase_id
    ''').fetchall()  # recuperar todos los detalles de las compras, incluyendo los items y sus cantidades
    conn.close()
    return [dict(purchase) for purchase in purchases]  # convertir cada compra en un diccionario y devolver la lista

def get_all_trips():
    conn = get_db_connection()
    trips = conn.execute('SELECT * FROM trips').fetchall()  # recuperar todos los viajes de la base de datos
    conn.close()
    return [dict(trip) for trip in trips]  # convertir cada viaje en un diccionario y devolver la lista


items = get_all_items()
print("Items:")
for item in items:
    print(item)

trucks = get_all_trucks()
print("\nTrucks:")
for truck in trucks:
    print(truck)

purchases = get_all_purchases()
print("\nPurchase Details:")
for purchase in purchases:
    print(purchase)

trips = get_all_trips()
print("\nTrips:")
for trip in trips:
    print(trip)


# función para imprimir los detalles de las asignaciones de camiones a viajes
def print_truck_assignments():
    conn = get_db_connection()  # obtener conexión a la base de datos
    trips = conn.execute('SELECT * FROM trips').fetchall()  # recuperar todos los viajes de la base de datos
    print("\nTrip Details:")  # imprimir detalles de los viajes
    for trip in trips:
        trip_weight = conn.execute('''
            SELECT SUM(items.weight * purchase_items.quantity) AS total_weight
            FROM purchase_items
            JOIN items ON purchase_items.item_id = items.item_id
            JOIN purchases ON purchase_items.purchase_id = purchases.purchase_id
            WHERE purchases.trip_id = ?
        ''', (trip['trip_id'],)).fetchone()['total_weight'] or 0  # calcular el peso total del viaje

        if trip['assigned_truck']:  # si el viaje tiene un camión asignado
            truck_capacity = conn.execute('''
                SELECT max_weight FROM trucks WHERE plate_number = ?
            ''', (trip['assigned_truck'],)).fetchone()['max_weight']  # obtener la capacidad máxima del camión asignado
            print(f"Trip {trip['trip_id']} to {trip['zip_codes']} has been assigned to truck {trip['assigned_truck']} (Capacity: {truck_capacity} tons). Trip weight: {trip_weight} tons.")
        else:
            print(f"Trip {trip['trip_id']} to {trip['zip_codes']} could not be assigned to truck (Capacity: {truck_capacity} tons). Trip weight: {trip_weight} tons.")  # imprimir que el viaje no pudo ser asignado a un camión
    conn.close()  # cerrar la conexión a la base de datos

print_truck_assignments()




Items:
{'item_id': 1, 'name': 'Navigation Log Pose', 'weight': 0.2, 'price': 300.0}
{'item_id': 2, 'name': 'Den Den Mushi', 'weight': 1.0, 'price': 150.0}
{'item_id': 3, 'name': 'Tony Chopper Hat', 'weight': 0.3, 'price': 15.0}
{'item_id': 4, 'name': "Roronoa Zoro's Swords Set", 'weight': 5.0, 'price': 500.0}
{'item_id': 5, 'name': 'Straw Hat', 'weight': 0.05, 'price': 20.0}
{'item_id': 6, 'name': 'Devil Fruit', 'weight': 0.2, 'price': 1000.0}
{'item_id': 7, 'name': 'Super Heavy Item 1', 'weight': 50.0, 'price': 2000.0}
{'item_id': 8, 'name': 'Super Heavy Item 2', 'weight': 100.0, 'price': 5000.0}

Trucks:
{'plate_number': 'MERRY GO', 'max_weight': 21.0, 'address': 'Dock #0, East Blue', 'working_days': 'Monday,Wednesday,Friday', 'available': 0}
{'plate_number': 'GOING MERRY', 'max_weight': 12.0, 'address': 'Dock #1, East Blue', 'working_days': 'Monday,Tuesday,Thursday', 'available': 0}
{'plate_number': 'THOUSAND SUNNY', 'max_weight': 1.0, 'address': 'Dock #2, Grand Line', 'working_days